# Test inattention detector

In [2]:
%cd inattention/

/home/michele/Desktop/Uni/sleep-detection-and-safety-pull-over/src/inattention


In [ ]:
# se non funziona: !conda install -c conda-forge libstdcxx-ng
from inattention.detector import InattentionDetector, WebcamCameraStream
import cv2
import time

cam = WebcamCameraStream()
detector = InattentionDetector(cam)

try:
    while True:
        # Get the latest frame (color, BGR)
        frame = cam.next()

        # Run detection asynchronously, and get last known result
        inattentive = detector.detect()

        status_text = "INATTENTIVE" if inattentive else "ATTENTIVE"
        color = (0, 0, 255) if inattentive else (0, 255, 0)  # red/green

        # Overlay text on frame
        cv2.putText(
            frame,
            status_text,
            (30, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.2,
            color,
            3,
            cv2.LINE_AA,
        )

        # Show the frame
        cv2.imshow("Driver Monitoring", frame)

        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

        # Small sleep to reduce CPU load
        time.sleep(0.08)

finally:
    # Cleanup
    detector.close()
    cam.close()
    cv2.destroyAllWindows()

Could not open camera device 0
[ WARN:0@24.828] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@24.828] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read frame from camera.
Failed to read

# Test RANSAC algorithm

In [37]:
import numpy as np
from pullover.utils import *

# ground-truth plane: 0.5 x + 0.2 y - 1 z + 1 = 0
a_true, b_true, c_true, d_true = 0.5, 0.2, -1.0, 1.0

# generate inliers
N_in = 200
rng = np.random.default_rng()
xs = rng.uniform(-10, 10, N_in)
ys = rng.uniform(-10, 10, N_in)
zs = -(a_true * xs + b_true * ys + d_true) / c_true  # exact
zs += rng.normal(scale=0.02, size=N_in) # add noise
inliers = np.vstack([xs, ys, zs]).T

# add random outliers
N_out = 100
outliers = rng.uniform(-10, 10, size=(N_out, 3))

pts = np.vstack([inliers, outliers])

plane, mask = fit_plane_ransac(pts, thresh=0.05, max_iterations=500, min_inliers=50, random_seed=1)
if mask is not None and plane is not None:
    print("plane:", plane)
    print("inliers:", int(mask.sum()), "of", pts.shape[0])
    print("distance origin-plane:", point_plane_distance(plane, (0,0,0)))


plane: (-0.4401640399356373, -0.17589109702843944, 0.8805214023143736, -0.8833426378031375)
inliers: 198 of 300
distance origin-plane: 0.8833426378031375
